Como comentamos anteriormente, hemoss apartado 10,000 filas del dataset original y las hemos guardado en el csv 'last_10000_rows' para hacer predicciones. Para hacer el pre procesamiento, pasamos la funcion `dataset_processing( )` al csv 'last_10000_rows'. Las variables de la función `dataset_processing( )` están en el Notebook de Preprocessing y están basadas en los datos de Train. Trabajamos en local con Jupyter Notebooks porque así se guardan las variables asociadas al preprocesamiento y esto permite que la función `dataset_processing( )` pueda acceder a ellas. Aplicamos la funcion al dataset y, al terminar, quitamos la columna `is_fraud`, que luego habrá que añadirla y poblarla a la hora de hacer predicciones.

In [1]:
# Some basic Data Libraries
import numpy as np
import pandas as pd

# All rows and columns display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_rows = pd.read_csv('./data/last_10000_rows.csv', sep=',', decimal='.')

In [3]:
df_rows.shape

(10000, 30)

In [4]:
#pip install scikit-learn --upgrade

Librerías que necesitamos en el Preprocesamiento:

In [5]:
from sklearn import preprocessing
!pip install geopy
from geopy.distance import geodesic

`dataset_processing( )` es la función que habíamos definido en el Notebook 4. Preprocesamiento

Pero primero tenemos que recuperar los valores de los Encoders que habíamos guardado, en previsión, en memoria:

In [6]:
%store -r le_gender
%store -r ohe_category
%store -r targ_job
%store -r ohe_state

In [7]:
def dataset_processing (df):

    # Dropping variables
    df = df.drop(['state_abbr','trans_date_trans_time', 'time', 'date','zip','city', 'cc_num'], axis = 1)

    # Weekdays into nums
    df['day_of_week'] = df['day_of_week'].map({
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6 })

    # is_holiday as num type
    df['is_holiday'] = df['is_holiday'].astype(int)

    # Label Encoder **previously defined in Train** for Gender
    df["gender"] = le_gender.transform(df["gender"])

    # One Hot encoder **previously defined in Train** for merchants categories
    cat_encoded = ohe_category.transform(df[["category"]])
    merch_cats = ohe_category.get_feature_names_out()
    df = pd.concat([df.drop(columns=["category"]), pd.DataFrame(cat_encoded.toarray(),columns=merch_cats)], axis=1)

    # Target encoder **previously defined in Train** for jobs
    job_encoded = targ_job.transform(df[['job']])
    te_job = targ_job.feature_names_in_
    df = pd.concat([df.drop(columns=["job"]), pd.DataFrame(job_encoded, columns=te_job)], axis=1)

    # One-Hot-Encoding **previously defined in Train** for State
    state_encoded = ohe_state.transform(df[["state"]])
    states = ohe_state.get_feature_names_out()
    df = pd.concat([df.drop(columns=["state"]), pd.DataFrame(state_encoded.toarray(), columns = states)], axis=1)

    # Distance
    df['merch_home_distance'] = df.apply(lambda row: geodesic((row['lat'], row['long']),
                                                          (row['merch_lat'], row['merch_long'])).miles, axis=1) #In miles
    # Online shopping transactions
    online_columns = [col for col in df.columns if col.endswith('_net')]
    df['is_online_shopping'] = df[online_columns].apply(lambda row: 1 if row.any() else 0, axis=1)

    return df

In [8]:
df_rows_processed = dataset_processing(df_rows)

In [10]:
df_rows_processed.shape

(10000, 88)

In [11]:
df_rows_processed = df_rows_processed.drop(['is_fraud'], axis=1)

In [12]:
df_rows_processed.shape

(10000, 87)

## Guardamos este fichero en otro csv:

In [13]:
df_rows_processed.to_csv('./data/last_10000_rows_proc.csv', index=False)